In [ ]:
%%file app.py

import json
from kafka import KafkaConsumer
from collections import defaultdict

KAFKA_TOPIC = 'urzedy'
SERVER = "broker:9092"

if __name__ == "__main__":
    consumer = KafkaConsumer(
        KAFKA_TOPIC,
        bootstrap_servers=[SERVER],
        value_deserializer=lambda x: json.loads(x.decode("utf-8")),
        auto_offset_reset='earliest',
        enable_auto_commit=True,
        api_version=(3, 7, 0)
    )

    total_waiting_time = defaultdict(int)
    message_count = defaultdict(int)
    results = []

    def save_results_to_file(results):
        with open('results.json', 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=4)

    try:
        for message in consumer:
            data = message.value
            office = data['Ulica']
            waiting_time = data['Czas_oczekiwania']
            
            total_waiting_time[office] += waiting_time
            message_count[office] += 1
            average_waiting_time = total_waiting_time[office] / message_count[office]

            result = {
                "Godzina": data['Godzina'],
                "Dzielnica": data['Dzielnica'],
                "Nazwa urzędu": f"Urząd Miasta Stołecznego Warszawa na ul. {data['Ulica']}",
                "Rodzaj sprawy": office,
                "Długość kolejki": data['Długość_kolejki'],
                "Liczba stanowisk": data['Liczba stanowisk'],
                "Czas oczekiwania": waiting_time,
                "Średni czas oczekiwania dla urzędu": average_waiting_time
            }
            
            results.append(result)

            # Print to console (optional)
            print(f"Godzina: {data['Godzina']}, Dzielnica: {data['Dzielnica']}, "
                  f"Nazwa urzędu: Urząd Miasta Stołecznego Warszawa na ul. {data['Ulica']}, "
                  f"Rodzaj sprawy: {office}, Długość kolejki: {data['Długość_kolejki']}, "
                  f"Liczba stanowisk: {data['Liczba stanowisk']}, Czas oczekiwania: {waiting_time} minut, "
                  f"Średni czas oczekiwania dla {office}: {average_waiting_time:.2f} minut")
            
            # Save the results to a JSON file after processing each message
            save_results_to_file(results)

    except KeyboardInterrupt:
        consumer.close()
        print("Consumer closed")


